In [160]:
# driver_service.py
from seleniumbase import Driver as SeleniumBaseDriver
from seleniumbase import BaseCase
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import undetected_chromedriver as uc
from selenium.common.exceptions import TimeoutException
import requests
from app.services.logger import Logger

class Driver(BaseCase):
    def __init__(self):
        # Configurar opções do Chrome
        self.options = uc.ChromeOptions()
        self.options.add_argument("--enable-automation")
        self.options.add_argument("--start-maximized")
        self.options.add_argument("--disable-notifications")
        self.options.add_argument("--disable-popup-blocking")
        self.options.add_argument("--kiosk-printing")
        self.options.add_argument("--disable-blink-features=AutomationControlled")
        self.options.add_argument("--no-sandbox")
        self.options.add_argument("--disable-dev-shm-usage")
        self.options.add_argument("--disable-popup-blocking")
        
        # Adicionar preferências para controlar PDFs
        prefs = {
            "download.prompt_for_download": False,
            "plugins.always_open_pdf_externally": False,
            "download.open_pdf_in_system_viewer": False,
            "download.default_directory": "/caminho/para/pasta/downloads"  # Substitua pelo caminho desejado
        }
        self.options.add_experimental_option("prefs", prefs)


        self.logger = Logger()
        
        # Remova ou comente a linha abaixo se não for usar o método add_extension para UC Mode

                
        # Inicializar o driver como None para que ele não seja iniciado
        self.driver = None

    def start_driver(self):
        if not self.driver:
            # Inicializar o driver do SeleniumBase com o modo UC e passando o diretório da extensão descompactada
            #self.driver = SeleniumBaseDriver(uc=True)
            self.driver = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=self.options)
            #self.driver.get("https://www.google.com")
            self.logger.log_info("Driver iniciado com sucesso")
        else:
            self.logger.log_info("Driver já está em execução")
        return self.driver
        
    def stop_driver(self):
        if self.driver:
            self.driver.quit()
            self.logger.log_info("Driver encerrado com sucesso")
        else:
            self.logger.log_info("Driver não está em execução")

    def safe_get_image_src(self, xpath):
        """Tenta obter o atributo src de uma imagem, retornando None se não existir."""
        elementos = self.find_elements(By.XPATH, xpath)
        return elementos[0].get_attribute("src") if elementos else None

    def get_session_id (self):
        return self.driver.session_id

    def disable_alert(self):
        self.driver.switch_to.alert.dismiss()

    def element_get_text(self, element, tag):
        if element in self.locator:
            try:
                # Aguardar até que o elemento seja visível e, em seguida, retornar seu texto
                element_text = self.wait.until(EC.visibility_of_element_located((self.locator[element], tag)))
                return element_text
            except TimeoutException:
                print("Elemento não encontrado")   
                  
    def get_elements(self, element, tag):
        if element in self.locator:
            try:
                # Aguardar até que o elemento seja visível e, em seguida, retornar seu texto
                elements = self.wait.until(EC.visibility_of_all_elements_located((self.locator[element], tag)))
                return elements
            except TimeoutException:
                print("Elemento não encontrado")

    def get(self, url):
        # await o.sleep(0)
        self.driver.get(url)
    def close(self):
    #  await o.sleep(0)
        self.driver.quit()   

    def close_session(self, session_id):
        grid_url = "https://grid.talentai.com.br/wd/hub"
        session_url = f"{grid_url}/session/{session_id}"
        response = requests.delete(session_url)
        if response.status_code == 200:
            print("Sessão fechada com sucesso!")
        else:
            print("Falha ao fechar a sessão.")

        return response    
    # Funcao para digitar no elemento           
    def sendkeys(self, element, tag, keys):
    #  await o.sleep(0)
        if element in self.locator:
            try:
                self.wait.until(EC.presence_of_element_located((self.locator[element], tag))).send_keys(keys)
            except TimeoutException:
                print("Elemento não encontrado")
                
    # Funcao para clicar no elemento                
    def click(self, element, tag):
    #  await asyncio.sleep(0)
        if element in self.locator:
            try:
                self.wait.until(EC.visibility_of_element_located((self.locator[element], tag))).click()
            except TimeoutException:    
                print("Elemento não encontrado")

In [161]:
username = "automacao"
password = "automacao558658"
site_url = "https://www.cartasdotarot.com.br/gerencial_sOAx0HTihLMl/"

In [162]:
self = Driver()
self.start_driver()

INFO - Driver iniciado com sucesso


<selenium.webdriver.remote.webdriver.WebDriver (session="d211050427ef388664bebfac90f9c354")>

In [163]:
def login(self, username, password, site_url):

    self.driver.get(site_url)
    self.driver.find_element(By.ID, "FLogin").send_keys(username)
    self.driver.find_element(By.ID, 'FSenha').send_keys(password)
    self.driver.find_element(By.XPATH, '/html/body/div/div[2]/form/button').click()


In [164]:
from app.services.driver import Driver
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from app.models.supa_connect import SupaConnect
from app.services.logger import Logger
from tqdm import tqdm


class ProfileActions:
    def __init__(self, logger=None):
        self.driver = Driver()
        self.supa = SupaConnect()
        self.logger = logger if logger else Logger()

    
    def login(self, username, password, site_url):

        try:
            self.driver.start_driver()
            self.driver.driver.get(site_url)
            time.sleep(1)
            self.driver.driver.find_element(By.ID, "FLogin").send_keys(username)
            self.driver.driver.find_element(By.ID, "FSenha").send_keys(password)
            self.driver.driver.find_element(By.XPATH, '/html/body/div/div[2]/form/button').click()

            time.sleep(3)

            return {'status': 'success'}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    
    def enter_atendentes(self, site_url):
        try:
            self.driver.driver.get(site_url + '/PG_Atendentes')
            self.driver.driver.find_element(By.ID, "select").click()
            self.driver.driver.find_element(By.XPATH, "/html/body/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[3]/th/table/tbody/tr/td[1]/select/option[9]").click()

            return {'status': 'success'}        
        except Exception as e:
            return {'status': 'error', 'message': str(e)}


    def get_checked_pix_tipo(self):
        """
        Encontra todos os inputs com name "PIXTipo" e itera sobre eles para
        retornar o valor do input selecionado.

        Parâmetros:
        - self: instância do Selenium Webself.

        Retorna:
        - O valor (atributo 'value') do input selecionado ou None se nenhum estiver marcado.
        """
        # Busca todos os inputs do tipo rádio com name "PIXTipo"
        pix_options = self.driver.driver.find_elements(By.XPATH, "//input[@name='PIXTipo']")
        
        # Itera sobre os elementos e retorna o que estiver selecionado
        for option in pix_options:
            if option.is_selected():
                return {"status": "success", "value": option.get_attribute('value')}
        
        return {"status": "error", "message": "Nenhum pix selecionado"}

    def get_checked_conta_tipo(self):
        """
        Encontra todos os inputs com name "Conta_Tipo", itera sobre eles para
        identificar o selecionado e retorna "Corrente" se o valor for "C" ou "Poupança" se for "P".

        Parâmetros:
        - self: instância do Selenium Webself.

        Retorna:
        - Uma string com o tipo de conta ou None se nenhum estiver marcado.
        """
        conta_options = self.driver.driver.find_elements(By.XPATH, "//input[@name='Conta_Tipo']")
        
        for option in conta_options:
            if option.is_selected():
                value = option.get_attribute("value")
                if value == "C":
                    return "Corrente"
                elif value == "P":
                    return "Poupança"
                else:
                    return value  # Para outros casos, se houver
        return None
    
    def table_to_dataframe(self):
        html_content = self.driver.driver.find_element(
            By.XPATH, '/html/body/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[4]/th'
        ).get_attribute('innerHTML')

        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the first table
        table = soup.find('table')

        # Check if the table was found
        if table:
            # Initialize a list to store the table data
            table_data = []
            # Iterate over rows in the table (<tr>)
            for row in table.find_all('tr'):
                row_data = []
                # Iterate over the cells (<td>)
                for cell in row.find_all(['td']):
                    value = cell.text.strip()
                    row_data.append(value if value else None)
                    # Check for a hyperlink
                    link = cell.find('a')
                    row_data.append(link.get('href') if link else None)
                if row_data:
                    table_data.append(row_data)

            # Create DataFrame from table_data
            df = pd.DataFrame(table_data)
            df.to_excel('arquivo.xlsx', index=False)

            # Uncomment or modify the following if you need to drop specific columns 
            # (using column indices might need adjustments based on the actual data structure)
            colunas_excluir = [1, 2, 3, 6, 7, 9, 11, 13, 14, 15, 16, 17, 18]
            df = df.drop(columns=colunas_excluir, errors='ignore')
            df = df.rename(columns={
                0: 'ID',
                4: 'Nome',
                5: 'Link',
                8: 'CPF',
                10: 'Creditos',
                12: 'Status',
                19: 'SiteVinculado',
            })
            df = df.drop(df.index[0])
            return df

        else:
            self.logger.error("No table found on the page.")
            return None

    def get_profile_infos(self, site_id, esoteric_site):
        elementos = self.driver.driver.find_elements(By.XPATH, "/html/body/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[6]/th/form/table/tbody/tr[56]/td[2]/img")
        foto = elementos[0].get_attribute("src") if elementos else None

        profile = {
            "nome": self.driver.driver.find_element(By.ID, "Nome").get_attribute("value"),
            "data_cadastro": self.driver.driver.find_element(By.ID, "DataCadastro").get_attribute("value"),
            "cpf": self.driver.driver.find_element(By.ID, "CPF").get_attribute("value"),
            "rg": self.driver.driver.find_element(By.ID, "RG").get_attribute("value"),
            "dt_nascimento": self.driver.driver.find_element(By.ID, "DataNascimento").get_attribute("value"),
            "email": self.driver.driver.find_element(By.ID, "Email").get_attribute("value"),
            "cep": self.driver.driver.find_element(By.ID, "En_CEP").get_attribute("value"),
            "logradouro": self.driver.driver.find_element(By.ID, "En_Endereco").get_attribute("value"),
            "complemento": self.driver.driver.find_element(By.ID, "En_Complemento").get_attribute("value"),
            "numero": self.driver.driver.find_element(By.ID, "En_Numero").get_attribute("value"),
            "bairro": self.driver.driver.find_element(By.ID, "En_Bairro").get_attribute("value"),
            "cidade": self.driver.driver.find_element(By.ID, "En_Cidade").get_attribute("value"),
            "uf": self.driver.driver.find_element(By.ID, "En_Estado").get_attribute("value"),
            "telefone": self.driver.driver.find_element(By.ID, "Telefone01").get_attribute("value"),
            "pix": self.driver.driver.find_element(By.ID, "PIX").get_attribute("value"),
            # Se desejar utilizar as funções já criadas para pegar o tipo, descomente as linhas abaixo:
            "tipo_pix": self.get_checked_pix_tipo(),
            "banco": self.driver.driver.find_element(By.ID, "Conta_Banco").get_attribute("value"),
            "agencia": self.driver.driver.find_element(By.ID, "Conta_Agencia").get_attribute("value"),
            "conta": self.driver.driver.find_element(By.ID, "Conta_Numero").get_attribute("value"),
            "favorecido": self.driver.driver.find_element(By.ID, "Conta_Favorecido").get_attribute("value"),
            "cpf_conta": self.driver.driver.find_element(By.ID, "Conta_CPF").get_attribute("value"),
            "tipo_conta": self.get_checked_conta_tipo(),
            "foto": foto,
            "site_nome": self.driver.driver.find_element(By.ID, "SiteNome").get_attribute("value"),
            "experiencia": self.driver.driver.find_element(By.ID, "Experiencia").get_attribute("value"),
            "horarioatendimento": self.driver.driver.find_element(By.ID, "HorarioAtendimento").get_attribute("value"),
            "oraculos": self.driver.driver.find_element(By.ID, "Oraculos").get_attribute("value"),
            "frase": self.driver.driver.find_element(By.ID, "Frase").get_attribute("value"),
            "sitedescricao": self.driver.driver.find_element(By.ID, "SiteDescricao").get_attribute("value"),
            "id_site": site_id,
            "esoteric_site": esoteric_site,
            "status": False
        }
        return profile

    def profiles_exist(self, **kwargs):

        print(kwargs["id_site"], kwargs["esoteric_site"])

        profile = self.supa.get_profile(kwargs['id_site'], kwargs['esoteric_site'])

        return profile


    def iter_in_profiles(self, profiles, site_url, esoteric_site, **kwargs):
        profile_saved = []
        profile_updated = []
        controle = self.logger.controle
        
        # Adicionando tqdm para a barra de progresso
        for index, row in tqdm(profiles.iterrows(), total=len(profiles), desc="Processando perfis", unit="perfil"):
            self.logger.log_info(f"Processando perfil {row['ID']}")
            try:
                self.driver.get(site_url + "/PG_Atendentes/Pg.Edicao.php?Codigo=" + str(row['ID']))

                time.sleep(0.5)

                profile = self.get_profile_infos(site_id=row['ID'], esoteric_site=esoteric_site)

                profile_exists = self.profiles_exist(id_site=row['ID'], esoteric_site=esoteric_site)
                
                if len(profile_exists.data) > 0:
                    resp_update = self.supa.update_profiles(profile=profile, profile_id=row['ID'], esoteric_site=esoteric_site)
                    profile_updated.append(resp_update.data[0]["id_site"])
                    self.logger.log_info(f"Perfil {row['ID']} atualizado com sucesso!")
                else:
                    resp_save = self.supa.save_profiles(profile=profile)
                    profile_saved.append(resp_save.data[0]["id_site"])
                    self.logger.log_info(f"Perfil {row['ID']} salvo com sucesso!")
            except Exception as e:
                self.logger.log_error(f"Erro ao processar perfil {row['ID']}: {e}")
                continue

        self.logger.log_info(f"Total de perfis salvos: {len(profile_saved)}")
        self.logger.log_info(f"Total de perfis atualizados: {len(profile_updated)}")

        self.supa.controle_update(controle_id=controle, payload={'status': 'Concluído'})
        return profile_saved




In [165]:
login(self, username, password, site_url)

In [183]:

def table_to_dataframe(html_content):

    soup = BeautifulSoup(html_content, 'html.parser')

    # Encontra a tabela desejada (selecionando-a pela classe, id ou outras características)
    table = soup.find('table')

    # Verifica se a tabela foi encontrada
    if table:
        # Inicializa uma lista para armazenar os dados da tabela
        table_data = []
        # Itera sobre as linhas da tabela (<tr>)
        for row in table.find_all('tr'):
            # Inicializa uma lista para armazenar os dados de uma linha
            row_data = []
            # Itera sobre as células da linha (<td>)
            for cell in row.find_all(['td']):
                # Adiciona o texto da célula à lista de dados da linha
                value = cell.text.strip()
                # Verifica se o valor não está vazio
                if value:
                    row_data.append(value)
                else:
                    row_data.append(None)
                # Verifica se a célula contém uma tag de âncora (hiperlink)
                link = cell.find('a')
                if link:
                    # Se houver uma tag de âncora, adiciona o link (href) à lista de dados da linha
                    row_data.append(link.get('href'))
                else:
                    row_data.append(None)
            # Adiciona os dados da linha à lista de dados da tabela
            if row_data:
                table_data.append(row_data)

        # Imprime os dados da tabela
        
        df = pd.DataFrame(table_data)
        df.to_excel('arquivo.xlsx', index=False)

        return df

def converter_credito_para_float(valor):
    import re
    try:
        valor = valor.replace('R$ ', '').strip()
        if '.' in valor and ',' in valor:
            valor = valor.replace('.', '').replace(',', '.')
        elif ',' in valor:
            valor = valor.replace(',', '.')
        numeros = re.findall(r'-?\d+\.?\d*', valor)
        return float(numeros[0]) if numeros else None
    except Exception as e:
        print(f"Erro ao converter crédito: {e}")
        return None

In [ ]:
def get_table_payments(self, site_id):
    self.driver.get(site_url + '/PG_Atendentes/Pg.Fechamento.php')
    time.sleep(2)
    table =  self.driver.find_element(By.XPATH, '/html/body/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[6]/th').get_attribute('innerHTML')
    data = table_to_dataframe(table)

    df = pd.DataFrame(data)
    df = df.rename(columns={
                0: 'ID',
                2: 'Nome',
                3: 'Link',
                4: 'CPF',
                6: 'Creditos',
            })
    df = df.drop(df.index[0])
    df.drop(df.columns[[1, 4]], axis=1, inplace=True)
    df['SiteVinculado'] = site_id

    for index, row in df.iterrows():
        valor_convertido = converter_credito_para_float(row['Creditos'])
        df.at[index, 'Creditos'] = valor_convertido

    return df

     ID                   Nome                   Link     5 Creditos     7  \
1    15       Agatha Sensitiva  Pg.Edicao.php?Codigo=  None     44.2  None   
2    18              Aylla Luz  Pg.Edicao.php?Codigo=  None     49.6  None   
3    19  Consultas Avaliativas  Pg.Edicao.php?Codigo=  None     -3.5  None   
4    23          Cigana Helena  Pg.Edicao.php?Codigo=  None      0.0  None   
5    26                 Anitta  Pg.Edicao.php?Codigo=  None    249.6  None   
..  ...                    ...                    ...   ...      ...   ...   
69  283       Cartomante Paula  Pg.Edicao.php?Codigo=  None     6.24  None   
70  284         Cigano Salomão  Pg.Edicao.php?Codigo=  None    48.96  None   
71  285          Áquila Cigana  Pg.Edicao.php?Codigo=  None      0.0  None   
72  286       Hadja do deserto  Pg.Edicao.php?Codigo=  None      4.8  None   
73  287        Bruxa Morrighan  Pg.Edicao.php?Codigo=  None     16.4  None   

   SiteVinculado  
1              1  
2              1  
3     

In [180]:
for index, row  in df.iterrows():
    print(row['Nome'])

Agatha Sensitiva
Aylla Luz
Consultas Avaliativas
Cigana Helena
Anitta
Inaê
Castiel
testeatt1 - NÃO REMOVER
Carmencita
Cigana Clarissa
Merlia
Cigano Belmiro
Cigana Carmem
Alice
Natasha Cigana
Nana
Adara
Bruxa Agnes
Amabile
Aretusa Cigana
Mística Hanna
Artemis
Cigana Sophia
Alba Cigana
Cigana Estrela da Noite
Cigana Clara
Mercúrio
Asa Mística
Bruxa Zoraide
Andrina Cigana
Amy
Apolo
Anay
Cigano Tássio
André
Bia de Padilha
Mistica Anny
Alita Cigana
Cigana Manuella
Cigana Flor
Cigana Sol
Bruxa Lilith
Bruxa de Mulambo
Cigana Jezabel
Bruxa Pandora
Sensitiva Lorran
Ayra
Octavia
Bruxa Bianca
Bruxa Sabrina
Cassandra
Adália
Mestre Gerson
Alana
Cigana Mariah
Dharana
Idalina Sensitiva
Ana Rosa
Acácia Cigana
Cigano Antônio
Mana Ma
Dalila
Sacerdotisa Vênus
Morgana
Mãe Ana de Oxum
Cigana Hana
Cigana Aurora
Lyah
Cartomante Paula
Cigano Salomão
Áquila Cigana
Hadja do deserto
Bruxa Morrighan


In [ ]:
import pandas as pd
profile_actions = ProfileActions()


def get_table_element(self, element, tag):
    try:
        # Obter o conteúdo HTML da tag <tbody>
        html_content = self.wait.until(EC.visibility_of_element_located((self.locator[element], tag))).get_attribute('innerHTML')
        print(html_content)
        # Extrair dados da tabela e transforma em dataframe
        data = profile_actions.table_to_dataframe(html_content)
        qtd_linhas = len(data)
        return data, qtd_linhas
    except TimeoutException:
        print("Elemento não encontrado")

def format_payments_table(self, site_id):
    """
    Formata a tabela de pagamentos extraída do site e retorna um DataFrame com os dados processados.
    """
    try:
        table =  self.driver.find_element(By.XPATH, '/html/body/table/tbody/tr[3]/td/table/tbody/tr/td/table/tbody/tr[6]/th').get_attribute('innerHTML')

        data = self.profile_actions.table_to_dataframe(table)
        
        df = pd.DataFrame(table[0])
        df = df.rename(columns={
            0: 'ID',
            2: 'Nome',
            3: 'Link',
            4: 'CPF',
            6: 'Creditos',
        })
        df = df.drop(df.index[0])
        # Remove colunas desnecessárias
        df.drop(df.columns[[1, 4]], axis=1, inplace=True)
        df['SiteVinculado'] = site_id
        return df
    except Exception as e:
        print(f"Erro ao formatar a tabela de pagamentos: {e}")

format_payments_table(self, 1) 

In [135]:
import asyncio
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Suponha que as funções get_table_element e table_to_dataframe já estejam definidas conforme a implementação anterior.

async def main():
    # Inicializa o driver (certifique-se de ter o driver adequado instalado)
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    
    # Exemplo de locator: ajuste conforme o elemento que deseja buscar
    locator = {
        'minha_tabela': (By.ID, 'id_da_tabela')  # Exemplo: procura um elemento pelo ID
    }
    
    # Parâmetros para a chamada: 
    element = 'minha_tabela'
    tag = 'tbody'  # Exemplo: estamos buscando a tag <tbody> dentro do elemento
    
    # Chama a função de forma assíncrona
    df, qtd_linhas = await get_table_element(element, tag, wait, locator)
    
    if df is not None:
        print("DataFrame criado com sucesso!")
        print("Quantidade de linhas:", qtd_linhas)
        print(df)
    else:
        print("Falha ao obter os dados da tabela.")
    
    # Finaliza o driver
    driver.quit()

# Executa a função principal de forma assíncrona
if __name__ == '__main__':
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop